In [15]:
import pandas as pd
### pval <= 5e-8
gwas_results = pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/sod1_project/variant_file/vanRheene_2021_summary_stats_hg38.csv")


In [ ]:
lead_variants=[['chr9',27563868,'rs2453555'],
              ['chr19',17752689,'rs12608932'],
              ['chr21',33039603,'rs80265967'],
              ['chr14',31045596,'rs229195'],
              ['chr3',39508968,'rs631312'],
              ['chr6',32672641,'rs9275477'],
              ['chr12',57975700,'rs113247976'],
              ['chr21',45753117,'rs75087725'],
              ['chr5',150410835,'rs10463311'],
               ['chr20',48438761,'rs17785991'],
               ['chr12',64877053,'rs4075094'],
               ['chr5',172354731,'rs517339'],
               ['chr4',170583157,'rs62333164'],
               ['chr13',46113984,'rs2985994'],
               ['chr7',157481780,'rs10280711']]

In [5]:
import pyfaidx
genome = pyfaidx.Fasta("/oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/hg38/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta")


In [ ]:
total_filtered_results=[]
for chromo,hg17_pos,rsid in lead_variants:
    row = gwas_results[gwas_results['rsid'] == rsid]
    assert chromo == row['chr'].values
    assert hg17_pos == row['pos'].values
    
    pos = row['start'].values[0]
    pos = pos - 1
    ref = row['ref'].values
    alt = row['alt'].values
    assert genome[chromo][pos:pos+1].seq == ref or genome[chromo][pos:pos+1].seq == alt
    
    
    ### get all variants +/- 250kb from lead variants
    start = pos - 250000
    end = pos + 250000
    filtered_results = gwas_results[(gwas_results['chr'] == chromo) & (gwas_results['start'] > start) & (gwas_results['end'] < end)]
    total_filtered_results += [filtered_results]

In [ ]:
total_filtered_results=pd.concat(total_filtered_results)

In [ ]:
total_filtered_results = total_filtered_results.drop_duplicates()

In [ ]:
total_filtered_results=total_filtered_results[['chr','start','rsid','ref','alt']]

In [30]:
total_filtered_results.to_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/sod1_project/variant_file/vanRheene_2021_250kb_from_lead_formatted.tsv",\
                   sep='\t',\
                   header=None,
                   index=False)

In [32]:
len(total_filtered_results) * 10

295750

### merge logfc across folds

In [1]:
import pandas as pd
total_variant_scores=[]
for fold in range(5):
    variant_scores = pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/variant_scoring/alpha_control/vanRheene_2021_250kb_from_lead_fold%s.variant_scores.tsv"%fold,\
           sep='\t')
    total_variant_scores += [variant_scores[['chr','pos','variant_id','allele1','allele2','allele1_pred_counts','allele2_pred_counts','logfc','abs_logfc','jsd']]]

In [2]:
total_variant_scores=pd.concat(total_variant_scores).groupby(['chr','pos','variant_id','allele1','allele2']).mean()

In [3]:
total_variant_scores=total_variant_scores.reset_index()

### some test 

In [14]:
for i, row in total_variant_scores.iterrows():
    chromo = row['chr']
    pos = row['pos']-1
    rsid = row['variant_id']
    ref = row['allele1']
    alt = row['allele2']
    assert genome[chromo][pos].seq == ref or genome[chromo][pos].seq == alt

In [33]:
from tqdm import tqdm
for i, row in tqdm(total_variant_scores.iterrows()):
    chromo = row['chr']
    pos = row['pos']-1
    rsid = row['variant_id']
    ref = row['allele1']
    alt = row['allele2']

    gwas_row=gwas_results[gwas_results['rsid'] == rsid]
    assert gwas_row['chr'].values == chromo
    assert gwas_row['start'].values[0] == pos+1

200it [01:50,  1.81it/s]


KeyboardInterrupt: 

In [44]:
total_variant_scores.to_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/output/variant_scoring/alpha_control/vanRheene_2021_250kb_from_lead_averaged.variant_scores.tsv",\
                           sep='\t',index=False)